# Set-up

In [14]:
import os
import base64
from langchain.indexes import VectorstoreIndexCreator
from impersonate_gpt.utils.langchain_utils import get_actual_prompt
from langchain.callbacks import get_openai_callback
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [2]:
from langchain.document_loaders import WhatsAppChatLoader

# Lanchaing creation

In [3]:
# OpenAI saved encoded in base64 in a single-line file at data folder
with open("../data/secrets.txt") as secrets:
    open_ai_key = secrets.readline()
    open_ai_key = base64.b64decode(open_ai_key).decode('ascii')
os.environ["OPENAI_API_KEY"] = open_ai_key

In [19]:
loader = WhatsAppChatLoader("../data/andrea_wchat.txt")
docs = loader.load()

In [27]:
docs

[Document(page_content='Andrea Paricio Henares on 19/10/16, 12:38: Hola, soy Andrea, de la facultad de bellas artes. Una pregunta, hay alguna posibilidad para poder ir a Granada el jueves en vez del miércoles?\n\nAitor Mira on 19/10/16, 13:24: Buenas Andrea!\n\nAitor Mira on 19/10/16, 13:24: Si\n\nAitor Mira on 19/10/16, 13:24: Hay posibilidad\n\nAitor Mira on 19/10/16, 13:25: Tu compañera micaela me ha comentado lo mismo\n\nAitor Mira on 19/10/16, 13:25: y supongo que yo también tendré que ir jueves\n\nAndrea Paricio Henares on 19/10/16, 13:26: Pues ya me vas diciendo :)\n\nAndrea Paricio Henares on 19/10/16, 13:26: Si necesitáis ayuda con algo también\n\nAitor Mira on 19/10/16, 13:27: No te preocupes, está todo bajo control. Gracias :)\n\nAitor Mira on 21/10/16, 13:29: Buenas de nuevo Andrea\n\nAitor Mira on 21/10/16, 13:30: el jueves por la mañana podrías salir, no?\n\nAndrea Paricio Henares on 21/10/16, 14:24: Sí\n\nAitor Mira on 24/10/16, 11:16: Buenas Andrea,\n\nAndrea Paricio He

In [20]:
# Creating the cv database for document indexing
index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


In [30]:
retriever = docsearch.vectorstore.as_retriever()

In [32]:
retriever.get_relevant_documents("test")

[Document(page_content='Aitor Mira on 14/11/16, 21:31: Eso digo\n\nAitor Mira on 14/11/16, 21:31: Pero al final los efectivos son los mismos\n\nAitor Mira on 14/11/16, 21:31: Y para lo que me queda\n\nAitor Mira on 14/11/16, 21:31: No puedo reestructurar\n\nAndrea Paricio Henares on 14/11/16, 21:32: la normativa está para saltarsela (?)\n\nAndrea Paricio Henares on 14/11/16, 21:32: o eso dice Yolanda\n\nAitor Mira on 14/11/16, 21:32: Jajajajajaja\n\nAitor Mira on 14/11/16, 21:33: No es problema normativo\n\nAitor Mira on 14/11/16, 21:33: Es cuestión de recursos humanos\n\nAitor Mira on 14/11/16, 21:33: Y de situación\n\nAitor Mira on 14/11/16, 21:34: Con una candidatura de daupv gestandose, nosotros centrados en cerrar proyectos, y los exámenes por venir\n\nAitor Mira on 14/11/16, 21:34: Poco margen me deja\n\nAndrea Paricio Henares on 14/11/16, 21:35: es más bien un problema de que no puedes meterle tantas cosas en la cabeza a una persona en tan poco tiempo\n\nAndrea Paricio Henares o

In [21]:
# Creating the langchain with the default OpenAI model for quering
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=docsearch.vectorstore.as_retriever(),
    input_key="question",
    verbose=True,
    return_source_documents=True,
)

# Playing with chain

In [23]:
# Simple question
question = "Who is Aitor Mira Abad?"
response = chain({"question": question})
print(response["result"])



> Entering new RetrievalQA chain...

> Finished chain.
 Aitor Mira Abad appears to be a student at a university who was involved in student representation.


In [24]:
# Retrieving the prompt sent to OpenAI LLM. Interesting for understanding how langchain works.
question = "Who is Aitor Mira Abad?"
actual_prompt = get_actual_prompt(chain, question)
print(actual_prompt)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Aitor Mira on 15/11/16, 14:50: 😊

Aitor Mira on 15/11/16, 18:03: Laura is out

Andrea Paricio Henares on 15/11/16, 18:32: De central tb?

Aitor Mira on 15/11/16, 18:33: De todo

Andrea Paricio Henares on 15/11/16, 18:34: Qué bien

Andrea Paricio Henares on 15/11/16, 18:34: Seguro que a vosotros no os ha hecho discurso

Aitor Mira on 15/11/16, 18:36: Bueno chicos, chicas, chiques, demomento no voy a seguir en la representación estudiantil, creo que ha llegado mi hora y aunque me gustaría seguir trabajando en este proyecto estoy aprendiendo a delegar y estoy un momento de hacer solo las cosas para mí misma. Acordaros de ver más allá de las ingenierías, de ver BBAA como otra carrera técnica más que aunque no llevemos el ingeniería delante hacemos muchas cosas útiles y lo estamos demostrando. Tenéis mi número para lo que sea :).

In [25]:
# Getting the retrieved docs for query
chain._get_docs(question)

[Document(page_content='Aitor Mira on 15/11/16, 14:50: 😊\n\nAitor Mira on 15/11/16, 18:03: Laura is out\n\nAndrea Paricio Henares on 15/11/16, 18:32: De central tb?\n\nAitor Mira on 15/11/16, 18:33: De todo\n\nAndrea Paricio Henares on 15/11/16, 18:34: Qué bien\n\nAndrea Paricio Henares on 15/11/16, 18:34: Seguro que a vosotros no os ha hecho discurso\n\nAitor Mira on 15/11/16, 18:36: Bueno chicos, chicas, chiques, demomento no voy a seguir en la representación estudiantil, creo que ha llegado mi hora y aunque me gustaría seguir trabajando en este proyecto estoy aprendiendo a delegar y estoy un momento de hacer solo las cosas para mí misma. Acordaros de ver más allá de las ingenierías, de ver BBAA como otra carrera técnica más que aunque no llevemos el ingeniería delante hacemos muchas cosas útiles y lo estamos demostrando. Tenéis mi número para lo que sea :). Muchos ánimos y mucha suerte en lo que os devenga.\n\nAitor Mira on 15/11/16, 18:36: este mensaje repetido en todos los grupos 

In [26]:
# More elaborated question with the OpenAI returning some useful execution information
with get_openai_callback() as cb:
    query = "What kind of relationship does Aitor have with Andrea?"
    response = chain({"question": query})
    print(response["result"])
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")




> Entering new RetrievalQA chain...

> Finished chain.
 Aitor and Andrea are friends.
Total Tokens: 1634
Prompt Tokens: 1627
Completion Tokens: 7
Successful Requests: 1
Total Cost (USD): $0.03268
